In [ ]:
# Import dependencies

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite

engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model

Base = automap_base()

# reflect the tables

Base.prepare(engine, reflect=True) 

In [ ]:
# View all of the classes that automap found

Base.classes.keys() 

In [ ]:
# Save references to each table

Measurement = Base.classes.measurement

Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB

session = Session(engine) 

## Bonus Challenge Assignment: Temperature Analysis II

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates

def calc_temps(start_date, end_date):
    
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example

print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set

# NB: I calculated for all years in the dataset

print(f'2017 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2017-08-01", "2017-08-07")}')
print(f'2016 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2016-08-01", "2016-08-07")}')
print(f'2015 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2015-08-01", "2015-08-07")}')
print(f'2014 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2014-08-01", "2014-08-07")}')
print(f'2013 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2013-08-01", "2013-08-07")}')
print(f'2012 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2012-08-01", "2012-08-07")}')
print(f'2011 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2011-08-01", "2011-08-07")}')
print(f'2010 temps 8-1 to 8-7 (min, avg, max): {calc_temps("2010-08-01", "2010-08-07")}')

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

aug2017 = calc_temps("2017-08-01", "2017-08-07")
aug2017_min = aug2017[0][0]
aug2017_avg = aug2017[0][1]
aug2017_max = aug2017[0][2]
aug2017_peak_to_peak = aug2017_max - aug2017_min

plt.bar(x="Aug2017", height=aug2017_avg, yerr=aug2017_peak_to_peak)

plt.title("Trip Avg Temp")
plt.ylabel("Temp (F)")
plt.ylim(0,100)
plt.grid(True)
plt.tight_layout()

plt.show()

### Daily Rainfall Average

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

stations_results = session.query(Station.station).distinct().all()
stations_list = [station[0] for station in stations_results]

print(stations_list)

In [ ]:
precip_list = []

for station_name in stations_list:
    
    # func.sum for this one

    sel = [func.sum(Measurement.prcp), Station.id, Station.name, Station.latitude, Station.longitude, Station.elevation]
    
    results = session.query(*sel).filter(Measurement.station == Station.station)\
        .filter(Measurement.station == station_name).filter(Measurement.date >= "2017-08-01")\
        .filter(Measurement.date <= "2017-08-07").all()

    
    if results[0][0] == None:
        
        continue
        
    else:
        
        precip_list.append(results)
    
for data in sorted(precip_list, reverse=True):
    
    print(data)

In [ ]:
precip_list = []

for station_name in stations_list:

    # func.avg for this one

    sel = [func.avg(Measurement.prcp), Station.id, Station.name, Station.latitude, Station.longitude, Station.elevation]
    
    results = session.query(*sel).filter(Measurement.station == Station.station)\
        .filter(Measurement.station == station_name).filter(func.strftime("%m-%d", Measurement.date) >= "08-01")\
        .filter(func.strftime("%m-%d", Measurement.date) <= "08-07").all()
        
    if results[0][0] == None:
        
        continue
        
    else:
        
        precip_list.append(results)
    
for data in sorted(precip_list, reverse=True):
    
    print(data)

### Daily Temperature Normals

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example

daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

normals = []

# Set the start and end date of the trip

start_date = '2017-08-01'
end_date = '2017-08-07'

# Use the start and end date to create a range of dates

start = dt.datetime.strptime(start_date, "%Y-%m-%d")
end = dt.datetime.strptime(end_date, "%Y-%m-%d")

date_range = pd.date_range(start, end)


# Strip off the year and save a list of strings in the format %m-%d

mon_day_list = date_range.strftime("%m-%d").tolist()

# print(mon_day_list)

# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.

for mon_day in mon_day_list:
       
    t_min, t_avg, t_max = daily_normals(mon_day)[0]
    normals.append(dict(t_min=t_min, t_avg=t_avg, t_max=t_max))    

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index

df = pd.DataFrame(data=normals, index=date_range)

df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`

plt.figure(figsize=(18,10))

df.plot.area(stacked=False)
plt.grid(True)

plt.xlabel("Date")
plt.ylabel("Temperature")
plt.ylim(0,85)
plt.xticks(ticks=df.index, labels=df.index.strftime("%Y-%m-%d"), rotation=45)

plt.tight_layout()

plt.show()

## Close Session

In [ ]:
session.close()